In [10]:
import torch
from torch import nn 
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
import time
import math
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

In [11]:
class NN(nn.Module):
    def __init__(self,):
        super(NN, self).__init__()
        #Linear layers
        self.mod = nn.Sequential(
            nn.Linear(1, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 20),
            torch.nn.Tanh(),
            nn.Linear(20, 1)
        )
        
    def forward(self, inpu):
        out = self.mod(inpu)
        return out

In [12]:
class Model:
    def __init__(self):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

        self.model = NN().to(device)
        
        #Max iteration
        self.max_iter = 1000
        
        #Criterion for loss 
        self.criterion = nn.MSELoss()
        
        #Optimizer
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=100,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps
        )
        
        #Initialization for training
        self.t_step = 0.01
        t = torch.arange(0, 15 + self.t_step, self.t_step) #create grid for t-axis #[0, 15] 1500
        print(t.shape)
        #Create grid for solution
        self.grid = t.reshape(1, -1).T.to(device)  # no need to create grid for 1D problem #collocations
        self.grid.requires_grad = True
        print(self.grid.shape)
        #Training data
        initial = torch.tensor([t[0]]) # Points for boundary conditions (both are same) #t=0, t=0, t=0, t=0, ...
        print(initial.shape)
        self.train_points = initial.reshape(1, -1).T.to(device) # Training points as a grid
        self.train_points.requires_grad = True
        print(self.train_points.shape)
        self.u_initial = torch.ones(len(initial)).reshape(1,-1).T.to(device) #1, 1, 1, 1
        self.ut_initial = -1*torch.ones(len(initial)).reshape(1,-1).T.to(device) #-1, -1, -1, -1, -1 ...
        print(self.u_initial.shape)
        print(self.ut_initial.shape)
        print(self.u_initial)
        print(self.ut_initial)
        print(self.train_points)
        """
        self.X = self.X.to(device)
        self.X_train = self.X_train.to(device)
        self.y_train = self.y_train.to(device)
        self.X.requires_grad = True
        """
        self.iter = 1


    
    def loss(self):
        self.optimizer.zero_grad()
        
        y = self.model(self.train_points) #0 taille 1500
        dy_dt = torch.autograd.grad(inputs=self.train_points, outputs=y, grad_outputs=torch.ones_like(y), retain_graph=True, create_graph=True)[0]
        
        loss = self.criterion(y, self.u_initial) #Compute first part of loss #should be a constant
        loss += self.criterion(dy_dt, self.ut_initial) #Compute second part of loss #should be a constant
        
        u = self.model(self.grid)
        #Compute gradients of grid with respect to t
        du_dt = torch.autograd.grad(inputs=self.grid, outputs=u, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        du_dt2 = torch.autograd.grad(inputs=self.grid, outputs=du_dt, grad_outputs=torch.ones_like(du_dt), retain_graph=True, create_graph=True)[0]

        loss += self.criterion(du_dt2 + 2*du_dt + u, torch.exp(-self.grid)*torch.cos(2*self.grid)) #third part of loss, using f(t,x) as in paper
        
        loss.backward()
        #Print loss every 100 iteration
        if self.iter % 100 == 0: 
            print(self.iter, loss.item())
        self.iter = self.iter + 1
        return loss

    def train(self):
        for i in range(self.max_iter):
            self.optimizer.step(self.loss)
        #self.lbfgs.step(self.loss)
    
    def predict(self):
        y_pred = self.model(self.grid)
        dy_dt = torch.autograd.grad(inputs=self.grid, outputs=y_pred, grad_outputs=torch.ones_like(y_pred), retain_graph=True, create_graph=True)[0]
        dy_dt2 = torch.autograd.grad(inputs=self.grid, outputs=dy_dt, grad_outputs=torch.ones_like(dy_dt), retain_graph=True, create_graph=True)[0]
        return dy_dt2 + 2*dy_dt + y_pred - torch.exp(-self.grid)*torch.cos(2*self.grid)

In [13]:
model = Model()
model.train()

torch.Size([1501])
torch.Size([1501, 1])
torch.Size([1])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 1])
tensor([[1.]], device='cuda:0')
tensor([[-1.]], device='cuda:0')
tensor([[0.]], device='cuda:0', requires_grad=True)


KeyboardInterrupt: 

In [ ]:
def result_funct(t):
    return (5/4)*torch.exp(-t) - (1/4)*torch.exp(-t)*torch.cos(2*t)

time_step = 0.01 #pourquoi /10 #0.001
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
time = torch.arange(0, 15 + time_step, time_step).reshape(1,-1).T.to(device)

f_pred = model.predict()
mod = model.model
mod.eval()
with torch.no_grad():
    y_pred = mod(time)#.cpu()
    #f_pred = 

y_pred = y_pred.detach().cpu()
time = time.detach().cpu()
mse = nn.MSELoss()
error = mse(y_pred, result_funct(time))

fig, ax = plt.subplots()
ax.plot(time, result_funct(time), color='black',linestyle='--', linewidth=2, label='True solution')
ax.plot(time, y_pred, color='blue', linestyle=':', linewidth=2, label='PINN estimation')
ax.legend()
plt.suptitle("Cauchy problem plotted")
plt.title("MSE of estimated solution: %f" %error, fontsize=8)
plt.xlabel("time (t)")
plt.ylabel("u(t)")
fig.show
print(error)

In [ ]:
y_pred.size()

In [ ]:
np.linalg.norm(y_pred - result_funct(torch.arange(0, 15 + 0.01, 0.01).reshape(1,-1).T),2)/np.linalg.norm(result_funct(torch.arange(0, 15 + 0.01, 0.01).reshape(1,-1).T),2) 

In [ ]:
fig, ax = plt.subplots()

ax.plot(torch.arange(0, 15 + 0.01, 0.01).reshape(1,-1).T, np.zeros(len(torch.arange(0, 15 + 0.01, 0.01).reshape(1,-1).T)), color='red', linewidth=2, label='DE true')
ax.plot(torch.arange(0, 15 + 0.01, 0.01).reshape(1,-1).T, f_pred.detach().cpu(), color='blue', linestyle=':', linewidth=2, label='DE')
fig.show 

In [ ]:
torch.mean((f_pred-torch.mean(f_pred,axis=0))**2,axis=0)[0]